<a href="https://colab.research.google.com/github/Iliasecc/Projets_Finance/blob/main/Pr%C3%A9diction_bourse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prédiction de bourse

In [6]:
!pip install keras

In [7]:
import yfinance as yf

# Définir le ticker de l'action (par exemple, Apple : AAPL)
ticker = 'AAPL'

# Télécharger les données historiques pour la période souhaitée
# Exemple : de 2023-01-01 à aujourd'hui
data = yf.download(ticker, start='2014-01-25', end='2024-01-25')

# Afficher les premières lignes des données téléchargées
print(data.head())

# Sauvegarder les données dans un fichier CSV si nécessaire
data.to_csv('historique_AAPL.csv')


[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open      Volume
Ticker           AAPL       AAPL       AAPL       AAPL        AAPL
Date                                                              
2014-01-27  17.152355  17.286334  17.004357  17.138958   554878800
2014-01-28  15.781410  16.046250  15.643379  15.851826  1065523200
2014-01-29  15.602251  15.808516  15.535886  15.701957   502810000
2014-01-30  15.572030  15.781412  15.476065  15.658026   678501600
2014-01-31  15.597581  15.626558  15.377919  15.428706   464797200


In [8]:
data1 = data[['Close']]

# Enregistrer le dataset filtré en local
data1.to_csv("aapl_historical_close_data.csv")

# 4. Effectuer une étude statistique sur les prix de clôture avec describe()
print("Statistiques descriptives sur les prix de clôture :")
print(data['Close'].describe())

Statistiques descriptives sur les prix de clôture :
Ticker         AAPL
count   2516.000000
mean      76.689691
std       56.936923
min       15.572030
25%       27.541584
50%       46.815128
75%      133.811958
max      197.144180


In [9]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import plotly.figure_factory as ff
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense


from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [10]:
# Charger les données depuis le fichier CSV
file_path = '/content/aapl_historical_close.csv'  # Remplace par le chemin réel vers ton fichier
df = pd.read_csv(file_path)
df['Date'] = df['Date'].apply(pd.to_datetime)
df.set_index('Date',inplace=True)
df.head()

,Close
Date,
2014-01-27,17.152351
2014-01-28,15.781413
2014-01-29,15.602255
2014-01-30,15.572035
2014-01-31,15.597575


In [11]:
# Créer un dataframe avec uniquement la colonne 'Close' pour la prédiction
new_df = pd.DataFrame()
new_df['Close'] = df['Close']
new_df.index = df.index

# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
final_dataset = new_df.values

# Diviser les données en ensemble d'entraînement et de validation
train_size = int(len(final_dataset) * 0.7)  # 70% pour l'entraînement, 20% pour la validation
train_data = final_dataset[:train_size]
valid_data = final_dataset[train_size:]

# Créer des dataframes pour l'entraînement et la validation
train_df = pd.DataFrame(train_data, columns=['Close'])
train_df.index = df[:train_size].index
valid_df = pd.DataFrame(valid_data, columns=['Close'])
valid_df.index = df[train_size:].index

# Normalisation des données (sur tout le dataset)
scaled_data = scaler.fit_transform(final_dataset.reshape(-1, 1))

# Création des ensembles d'entraînement
x_train_data, y_train_data = [], []

# Utilisation de fenêtres de 60 jours pour l'entraînement
for i in range(60, len(train_data)):
    x_train_data.append(scaled_data[i-60:i, 0])  # Fenêtres de 60 jours
    y_train_data.append(scaled_data[i, 0])  # Prix de la journée suivante

x_train_data, y_train_data = np.array(x_train_data), np.array(y_train_data)

# Reshape des données pour être compatibles avec LSTM (samples, timesteps, features)
x_train_data = np.reshape(x_train_data, (x_train_data.shape[0], x_train_data.shape[1], 1))

# Vérifier les dimensions
print("Shape of x_train_data:", x_train_data.shape)
print("Shape of y_train_data:", y_train_data.shape)

Shape of x_train_data: (1701, 60, 1)
Shape of y_train_data: (1701,)


In [28]:
# Créer le modèle LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train_data.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

lstm_model.compile(loss='mean_squared_error', optimizer='adam')

# Entraîner le modèle
lstm_model.fit(x_train_data, y_train_data, epochs=15, batch_size=3, verbose=2)

# Enregistrer le modèle après l'entraînement
model_path = "lstm_stock_market.h5"  # Spécifiez le chemin de sauvegarde
lstm_model.save(model_path)
print(f"Modèle sauvegardé sous le nom {model_path}")

# Préparation des données de test (validation)
inputs_data = new_df[len(new_df) - len(valid_data) - 60:].values  # Prendre les 60 derniers jours avant les données de validation
inputs_data = inputs_data.reshape(-1, 1)
inputs_data = scaler.transform(inputs_data)  # Normalisation


Epoch 1/15
567/567 - 20s - 36ms/step - loss: 6.1720e-04
Epoch 2/15
567/567 - 20s - 36ms/step - loss: 2.4593e-04
Epoch 3/15
567/567 - 20s - 36ms/step - loss: 1.7806e-04
Epoch 4/15
567/567 - 21s - 36ms/step - loss: 1.6100e-04
Epoch 5/15
567/567 - 20s - 36ms/step - loss: 1.8865e-04
Epoch 6/15
567/567 - 21s - 36ms/step - loss: 1.3396e-04
Epoch 7/15
567/567 - 20s - 36ms/step - loss: 9.9261e-05
Epoch 8/15
567/567 - 21s - 37ms/step - loss: 1.3759e-04
Epoch 9/15
567/567 - 20s - 35ms/step - loss: 8.7263e-05
Epoch 10/15
567/567 - 21s - 38ms/step - loss: 8.4629e-05
Epoch 11/15
567/567 - 20s - 35ms/step - loss: 9.1578e-05
Epoch 12/15
567/567 - 16s - 29ms/step - loss: 8.0707e-05
Epoch 13/15
567/567 - 21s - 37ms/step - loss: 8.6158e-05
Epoch 14/15
567/567 - 20s - 35ms/step - loss: 9.4648e-05
Epoch 15/15
567/567 - 21s - 37ms/step - loss: 1.0153e-04


Modèle sauvegardé sous le nom lstm_stock_market.h5


In [36]:

# Créer les fenêtres de 60 jours pour les données de validation
X_test = []

for i in range(60, inputs_data.shape[0]):
    X_test.append(inputs_data[i-60:i, 0])
print(X_test)
X_test = np.array(X_test)

# Reshaper pour la LSTM
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Prédiction sur X_test
predicted_closing_price = lstm_model.predict(X_test)

# Inverser la normalisation des prédictions
predicted_closing_price_normalise = scaler.inverse_transform(predicted_closing_price)

# Ajouter les prédictions à valid_df
valid_df['Predictions'] = predicted_closing_price_normalise

# Afficher les résultats
print(valid_df.head())


24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
                 Close  Predictions
Date                               
2021-01-25  139.755020   139.763641
2021-01-26  139.989716   143.091370
2021-01-27  138.914078   143.244614
2021-01-28  134.054153   141.982727
2021-01-29  129.037720   137.283447


In [37]:
valid_df.head()

,Close,Predictions
Date,,
2021-01-25,139.755020,139.763641
2021-01-26,139.989716,143.091370
2021-01-27,138.914078,143.244614
2021-01-28,134.054153,141.982727
2021-01-29,129.037720,137.283447


In [38]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df.index,y=train_df['Close'],
                    mode='lines',
                    name='Apple Train Data'))
fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Close'],
                    mode='lines',
                    name='Apple Valid Data'))
fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Predictions'],
                    mode='lines',
                    name='Prediction'))

In [39]:
from sklearn.metrics import mean_squared_error
print('The Mean Squared Error is',mean_squared_error(valid_df['Close'].values,valid_df['Predictions'].values))

The Mean Squared Error is 17.12706878471199


In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Assuming 'valid_df' has the actual prices in 'Close' column and the predicted prices in 'Predictions' column
# Replace 'Close' with the actual column name that holds the true values for your validation data IF it's not 'Close'.

# Actual values (True prices) and Predicted values (from your model)
y_true = valid_df['Close'].values  # Changed 'True Price' to 'Close'
y_pred = valid_df['Predictions'].values

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_true, y_pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# Calculate the R-squared (R²) value
r2 = r2_score(y_true, y_pred)
print(f"R-squared (R²): {r2:.2f}")

Mean Absolute Error (MAE): 3.42
Mean Squared Error (MSE): 17.13
Root Mean Squared Error (RMSE): 4.14
R-squared (R²): 0.96


In [41]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Close'],
                    mode='lines',
                    name='Test'))
fig.add_trace(go.Scatter(x=valid_df.index,y=valid_df['Predictions'],
                    mode='lines',
                    name='Predicted'))

In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Charger le modèle sauvegardé
model_path = "lstm_stock_market.h5"  # Chemin du modèle sauvegardé
lstm_model = load_model(model_path)
print(f"Modèle chargé depuis {model_path}")

# Fonction pour prédire le jour suivant
def predict_next_day(input_csv):
    # Charger les données depuis le fichier CSV d'entrée
    df = pd.read_csv(input_csv)
    df['Date'] = pd.to_datetime(df['Date'])  # Assurez-vous que la colonne 'Date' est bien au format datetime
    df.set_index('Date', inplace=True)

    # Vérifier si le fichier contient au moins 60 jours de données
    if len(df) < 60:
        print("Le fichier doit contenir au moins 60 jours de données pour faire une prédiction.")
        return None

    # Extraire les 60 derniers jours
    df_last_60_days = df[-60:]

    # Normalisation des données
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df_last_60_days['Close'].values.reshape(-1, 1))

    # Créer les données d'entrée pour la prédiction (les 60 derniers jours)
    x_input = scaled_data.reshape(1, 60, 1)

    # Prédiction sur la fenêtre de 60 jours
    predicted_next_day = lstm_model.predict(x_input)

    # Inverser la normalisation pour obtenir la valeur réelle de la prédiction
    predicted_next_day = scaler.inverse_transform(predicted_next_day)

    return predicted_next_day[0][0]

# Exemple d'appel de la fonction pour prédire le jour suivant
input_csv = '/content/corrigé - AAPL.csv'  # Remplace par le chemin réel de ton fichier CSV avec les données d'entrée
predicted_close = predict_next_day(input_csv)

# Si une prédiction a été faite, afficher et sauvegarder le résultat
if predicted_close is not None:
    # Afficher la prédiction du jour suivant
    print(f"Le prix de clôture prédit pour le jour suivant est : {predicted_close:.2f}")

    # Créer un DataFrame pour afficher la date suivante et le prix de clôture prédit
    next_day_date = pd.to_datetime(df.index[-1]) + pd.Timedelta(days=1)
    output_df = pd.DataFrame({
        'Date': [next_day_date],
        'Close': [predicted_close]
    })

    # Sauvegarder le résultat dans un fichier CSV
    output_csv = 'predicted_next_day.csv'
    output_df.to_csv(output_csv, index=False)

    print(f"Le résultat a été sauvegardé dans {output_csv}")


Modèle chargé depuis lstm_stock_market.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Le prix de clôture prédit pour le jour suivant est : 224.18
Le résultat a été sauvegardé dans predicted_next_day.csv


In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Charger le modèle sauvegardé
model_path = "lstm_stock_market.h5"  # Chemin du modèle sauvegardé
lstm_model = load_model(model_path)
print(f"Modèle chargé depuis {model_path}")

# Fonction pour prédire le jour suivant et ajouter les 60 jours utilisés à chaque prédiction
def predict_next_day(input_csv):
    # Charger les données depuis le fichier CSV d'entrée
    df = pd.read_csv(input_csv)
    df['Date'] = pd.to_datetime(df['Date'])  # Assurez-vous que la colonne 'Date' est bien au format datetime
    df.set_index('Date', inplace=True)

    # Vérifier si le fichier contient au moins 60 jours de données
    if len(df) < 60:
        print("Le fichier doit contenir au moins 60 jours de données pour faire une prédiction.")
        return None

    # Extraire les 60 derniers jours
    df_last_60_days = df[-60:]

    # Normalisation des données
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df_last_60_days['Close'].values.reshape(-1, 1))

    # Créer les données d'entrée pour la prédiction (les 60 derniers jours)
    x_input = scaled_data.reshape(1, 60, 1)

    # Prédiction sur la fenêtre de 60 jours
    predicted_next_day = lstm_model.predict(x_input)

    # Inverser la normalisation pour obtenir la valeur réelle de la prédiction
    predicted_next_day = scaler.inverse_transform(predicted_next_day)

    # Récupérer la prédiction
    predicted_close = predicted_next_day[0][0]

    # Afficher la prédiction du jour suivant
    print(f"Le prix de clôture prédit pour le jour suivant est : {predicted_close:.2f}")

    # Créer un DataFrame avec la date suivante et la prédiction
    next_day_date = pd.to_datetime(df.index[-1]) + pd.Timedelta(days=1)
    predicted_df = pd.DataFrame({
        'Date': [next_day_date],
        'Close': [predicted_close]
    })

    # Sauvegarder la prédiction dans un fichier CSV
    output_csv = 'predicted_next_day.csv'
    predicted_df.to_csv(output_csv, index=False)

    print(f"Le résultat de la prédiction a été sauvegardé dans {output_csv}")

    # Ajouter la prédiction à la fenêtre des 60 derniers jours et enregistrer la nouvelle fenêtre dans un fichier CSV
    # Utilisation de concat() pour ajouter la prédiction à la fenêtre
    new_window_1 = pd.concat([df_last_60_days, predicted_df], axis=0)
    new_window_1.to_csv('prediction_1.csv', index=True)

    print("La nouvelle fenêtre de 60 jours, incluant la prédiction, a été sauvegardée dans 'prediction_1.csv'.")

# Exemple d'appel de la fonction pour prédire le jour suivant
input_csv = '/content/final - eureka_AAPL.csv'  # Remplace par le chemin réel de ton fichier CSV avec les données d'entrée
predict_next_day(input_csv)


Modèle chargé depuis lstm_stock_market.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
Le prix de clôture prédit pour le jour suivant est : 194.35
Le résultat de la prédiction a été sauvegardé dans predicted_next_day.csv
La nouvelle fenêtre de 60 jours, incluant la prédiction, a été sauvegardée dans 'prediction_1.csv'.


In [71]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Charger le modèle sauvegardé
model_path = "lstm_stock_market.h5"  # Chemin du modèle sauvegardé
lstm_model = load_model(model_path)
print(f"Modèle chargé depuis {model_path}")

# Fonction pour prédire le jour suivant et ajouter les 60 jours utilisés à chaque prédiction
def predict_next_day(input_csv):
    # Charger les données depuis le fichier CSV d'entrée
    df = pd.read_csv(input_csv)
    df['Date'] = pd.to_datetime(df['Date'])  # Assurez-vous que la colonne 'Date' est bien au format datetime
    df.set_index('Date', inplace=True)

    # Vérifier si le fichier contient au moins 60 jours de données
    if len(df) < 60:
        print("Le fichier doit contenir au moins 60 jours de données pour faire une prédiction.")
        return None

    # Extraire les 60 derniers jours
    df_last_60_days = df[-60:]

    # Normalisation des données
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df_last_60_days['Close'].values.reshape(-1, 1))

    # Créer les données d'entrée pour la prédiction (les 60 derniers jours)
    x_input = scaled_data.reshape(1, 60, 1)

    # Prédiction sur la fenêtre de 60 jours
    predicted_next_day = lstm_model.predict(x_input)

    # Inverser la normalisation pour obtenir la valeur réelle de la prédiction
    predicted_next_day = scaler.inverse_transform(predicted_next_day)

    # Récupérer la prédiction
    predicted_close = predicted_next_day[0][0]

    # Afficher la prédiction du jour suivant
    print(f"Le prix de clôture prédit pour le jour suivant est : {predicted_close:.2f}")

    # Créer un DataFrame avec la date suivante et la prédiction
    next_day_date = pd.to_datetime(df.index[-1]) + pd.Timedelta(days=1)
    predicted_df = pd.DataFrame({
        'Date': [next_day_date],
        'Close': [predicted_close]
    })

    # Sauvegarder la prédiction dans un fichier CSV
    output_csv = 'predicted_next_day.csv'
    predicted_df.to_csv(output_csv, index=False)

    print(f"Le résultat de la prédiction a été sauvegardé dans {output_csv}")

    # Ajouter la prédiction à la fenêtre des 60 derniers jours et enregistrer la nouvelle fenêtre dans un fichier CSV
    # Utilisation de concat() pour ajouter la prédiction à la fenêtre
    new_window_2 = pd.concat([df_last_60_days, predicted_df], axis=0)
    new_window_2.to_csv('prediction_11.csv', index=True)

    print("La nouvelle fenêtre de 60 jours, incluant la prédiction, a été sauvegardée dans 'prediction_11.csv'.")

# Exemple d'appel de la fonction pour prédire le jour suivant
input_csv = '/content/prediction_10.csv'  # Remplace par le chemin réel de ton fichier CSV avec les données d'entrée
predict_next_day(input_csv)


Modèle chargé depuis lstm_stock_market.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 969ms/step
Le prix de clôture prédit pour le jour suivant est : 195.18
Le résultat de la prédiction a été sauvegardé dans predicted_next_day.csv
La nouvelle fenêtre de 60 jours, incluant la prédiction, a été sauvegardée dans 'prediction_11.csv'.


In [75]:
import pandas as pd
import plotly.graph_objects as go

# Charger les données réelles (comp1) et les prédictions (prediction_11)
comp1_path = '/content/traité - comp1_AAPL.csv'
prediction11_path = '/content/traité - prediction_11.csv'

comp1_df = pd.read_csv(comp1_path)
prediction11_df = pd.read_csv(prediction11_path)

# Convertir les dates au format datetime
comp1_df['Date'] = pd.to_datetime(comp1_df['Date'], format='%Y-%m-%d')  # Format de comp1
prediction11_df['Date'] = pd.to_datetime(prediction11_df['Date'], format='%d/%m/%Y')  # Format de prediction_11

# Créer une figure Plotly
fig = go.Figure()

# Tracer les prix réels (comp1)
fig.add_trace(go.Scatter(x=comp1_df['Date'], y=comp1_df['Close'],
                         mode='lines+markers', name='Prix Réels', line=dict(color='blue')))

# Tracer les prix prédits (prediction_11)
fig.add_trace(go.Scatter(x=prediction11_df['Date'], y=prediction11_df['Close'],
                         mode='lines+markers', name='Prix Prédit', line=dict(color='red')))

# Ajouter des titres et des légendes
fig.update_layout(
    title='Comparaison des prix réels et des prix prédits',
    xaxis_title='Date',
    yaxis_title='Prix de Clôture',
    legend_title="Légende",
    template='plotly_dark'  # Optionnel, pour un fond sombre
)

# Afficher la figure
fig.show()
